In [4]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 100

In [7]:
def stack_stats_table(df):
    """Lengthen into a series
    """
    stat = df.columns.levels[0][0]
    df.index = df.pop((stat, 'Player')).rename('player')
    
    if (stat, 'Tot') in df.columns:
        df = df.drop((stat, 'Tot'), 'columns')
    if 'Opponent' in df.index and 'Totals' in df.index:
        df = df.drop(['Opponent', 'Totals'])
        
    ser = df.stack().squeeze().rename_axis(['player', 'round'])
    return ser

def get_players_from_team_year(team, year):
    # Each table retrieved by pandas indicates a different table. These can be identified by the first level of the column index
    dfs = pd.read_html(f'https://afltables.com/afl/stats/teams/{team}/{year}_gbg.html')
    df = pd.concat((stack_stats_table(df) for df in dfs), 'columns')
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    df = df.assign(team=team, year=year).set_index(['team', 'year'], append=True).reorder_levels(['team', 'year', 'player', 'round'])
    try:
        df = df.replace('-', 0).astype(int)
    except:
        import ipdb; ipdb.set_trace()
    return df

teams = ['adelaide', 'brisbaneb', 'brisbanel', 'carlton', 'collingwood', 'essendon', 'fitzroy',
         'fremantle', 'geelong', 'goldcoast', 'gws', 'hawthorn', 'melbourne', 'kangaroos', 'padelaide',
         'stkilda', 'swans', 'university', 'westcoast', 'bullldogs']

from urllib.error import HTTPError

def get_all_players(years=(2019, ), teams=teams):
    dfs = []
    for y in years:
        for t in teams:
            try:
                df = get_players_from_team_year(t, y)
                dfs.append(df)
            except HTTPError:
                continue
    return pd.concat(dfs)

In [8]:
df = get_all_players(years=range(2019, 2020))
feats = df.columns.tolist()

In [52]:
grper = df.groupby(['team', 'year', 'round'])
dft = grper.sum()
dft['team_score'] = dft['goals'].mul(6) + dft['behinds']
dft['uncontested_marks'] = dft['marks'] - dft['contested_marks']
# scores = grper['goals'].sum().mul(6) + grper['behinds'].sum()

In [53]:
# df['team_score'] = grper['goals'].transform('sum').mul(6) + grper['behinds'].transform('sum')

In [ ]:
dft.head()

In [54]:
from sklearn.linear_model import Ridge

In [55]:
dft.head()

disposals  kicks  marks  handballs  goals  behinds  \
team     year round                                                       
adelaide 2019 R1           440    229     91        211      7        9   
              R2           377    226     93        151     12       14   
              R3           369    210     90        159     10       12   
              R4           355    213    109        142      8       10   
              R5           455    248    106        207     18       10   

                     hit_outs  tackles  rebounds  inside_50s  clearances  \
team     year round                                                        
adelaide 2019 R1           35       58        41          58          40   
              R2           47       63        47          52          38   
              R3           31       66        33          53          38   
              R4           21       52        45          36          19   
              R5           39       69        35          59          39   

                     clangers  frees  frees_against  contested_possessions  \
team     year round                                                          
adelaide 2019 R1           60     20             11                    172   
              R2           46     27             15                    137   
              R3           43     15             15                    115   
              R4           57     18             24                    120   
              R5           51     15             17                    169   

                     uncontested_possessions  contested_marks  \
team     year round                                             
adelaide 2019 R1                         265               11   
              R2                         229               11   
              R3                         235                9   
              R4                         240                7   
              R5                         283               15   

                     marks_inside_50  one_percenters  bounces  goal_assists  \
team     year round                                                           
adelaide 2019 R1                   8              57        3             5   
              R2                   7              46        4             5   
              R3                  12              51        7             8   
              R4                   9              42        3             6   
              R5                  11              50        6            11   

                     %_played  team_score  uncontested_marks  
team     year round                                           
adelaide 2019 R1         1801          51                 80  
              R2         1800          86                 82  
              R3         1797          72                 81  
              R4         1802          58                102  
              R5         1798         118                 91

In [62]:
import requests
r = requests.get('https://supercoach.heraldsun.com.au/2019/api/afl/classic/v1/players?round=7&embed=notes,odds,player_stats,positions&xredir=1')
j = r.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [63]:
r.text

''

In [59]:
model = Ridge()
feats = ['contested_marks', 'contested_possessions', 'uncontested_marks', 'frees', 'frees_against', 'clangers', 'one_percenters', 'hit_outs']
model.fit(dft[feats], dft['team_score'])
pd.Series(model.coef_, index=feats).mul(10)

contested_marks           1.029589
contested_possessions     6.750021
uncontested_marks         3.735641
frees                     4.533141
frees_against            15.807020
clangers                 -8.048450
one_percenters           -2.135229
hit_outs                 -1.282531
dtype: float64

In [26]:
df.head()

disposals  kicks  marks  handballs  goals  \
team     year player       round                                              
adelaide 2019 Atkins, Rory R1            31     22      7          9      0   
                           R2            21     14      3          7      1   
                           R3            12      6      3          6      1   
                           R4            21     17      6          4      0   
                           R5            26     17      5          9      0   

                                  behinds  hit_outs  tackles  rebounds  \
team     year player       round                                         
adelaide 2019 Atkins, Rory R1           1         0        0         2   
                           R2           1         0        1         4   
                           R3           1         0        0         3   
                           R4           0         0        1         4   
                           R5           0         0        5         6   

                                  inside_50s  clearances  clangers  frees  \
team     year player       round                                            
adelaide 2019 Atkins, Rory R1             10           1         7      0   
                           R2              4           2         1      0   
                           R3              2           2         0      0   
                           R4              2           1         2      0   
                           R5              4           4         3      0   

                                  frees_against  contested_possessions  \
team     year player       round                                         
adelaide 2019 Atkins, Rory R1                 3                      6   
                           R2                 0                      6   
                           R3                 0                      3   
                           R4                 0                      4   
                           R5                 0                     11   

                                  uncontested_possessions  contested_marks  \
team     year player       round                                             
adelaide 2019 Atkins, Rory R1                          26                0   
                           R2                          13                0   
                           R3                           8                0   
                           R4                          16                0   
                           R5                          16                0   

                                  marks_inside_50  one_percenters  bounces  \
team     year player       round                                             
adelaide 2019 Atkins, Rory R1                   0               0        1   
                           R2                   0               5        0   
                           R3                   0               0        0   
                           R4                   0               1        1   
                           R5                   0               0        1   

                                  goal_assists  %_played  team_score  
team     year player       round                                      
adelaide 2019 Atkins, Rory R1                0        88          51  
                           R2                0        87          86  
                           R3                0        77          72  
                           R4                0        87          58  
                           R5                1        85         118

In [23]:
# df.head()

In [24]:
# grper['behinds'].sum()

In [20]:
grper['goals'].sum()

team       year  round
adelaide   2019  R1        7
                 R2       12
                 R3       10
                 R4        8
                 R5       18
                 R6       15
                 R7        7
                 R8       13
brisbanel  2019  R1       15
                 R2       16
                 R3       16
                 R4        9
                 R5        8
                 R6       16
                 R7       14
                 R8       11
bullldogs  2019  R1       11
                 R2       16
                 R3        9
                 R4        9
                 R5        7
                 R6        9
                 R7       15
                 R8       13
carlton    2019  R1        9
                 R2       11
                 R3       10
                 R4        8
                 R5       15
                 R6       13
                          ..
padelaide  2019  R3       13
                 R4       14
                 R5 

In [16]:
scores

team       year  round
adelaide   2019  R1        51
                 R2        86
                 R3        72
                 R4        58
                 R5       118
                 R6        97
                 R7        50
                 R8        85
brisbanel  2019  R1       101
                 R2       105
                 R3       105
                 R4        62
                 R5        57
                 R6       110
                 R7        97
                 R8        73
bullldogs  2019  R1        80
                 R2       104
                 R3        65
                 R4        62
                 R5        55
                 R6        66
                 R7        99
                 R8        90
carlton    2019  R1        62
                 R2        70
                 R3        71
                 R4        56
                 R5       100
                 R6        85
                         ... 
padelaide  2019  R3        87
                 

In [6]:
df.head()

disposals  kicks  marks  handballs  goals  \
team     year player       round                                              
adelaide 2019 Atkins, Rory R1            31     22      7          9      0   
                           R2            21     14      3          7      1   
                           R3            12      6      3          6      1   
                           R4            21     17      6          4      0   
                           R5            26     17      5          9      0   

                                  behinds  hit_outs  tackles  rebounds  \
team     year player       round                                         
adelaide 2019 Atkins, Rory R1           1         0        0         2   
                           R2           1         0        1         4   
                           R3           1         0        0         3   
                           R4           0         0        1         4   
                           R5           0         0        5         6   

                                  inside_50s  clearances  clangers  frees  \
team     year player       round                                            
adelaide 2019 Atkins, Rory R1             10           1         7      0   
                           R2              4           2         1      0   
                           R3              2           2         0      0   
                           R4              2           1         2      0   
                           R5              4           4         3      0   

                                  frees_against  contested_possessions  \
team     year player       round                                         
adelaide 2019 Atkins, Rory R1                 3                      6   
                           R2                 0                      6   
                           R3                 0                      3   
                           R4                 0                      4   
                           R5                 0                     11   

                                  uncontested_possessions  contested_marks  \
team     year player       round                                             
adelaide 2019 Atkins, Rory R1                          26                0   
                           R2                          13                0   
                           R3                           8                0   
                           R4                          16                0   
                           R5                          16                0   

                                  marks_inside_50  one_percenters  bounces  \
team     year player       round                                             
adelaide 2019 Atkins, Rory R1                   0               0        1   
                           R2                   0               5        0   
                           R3                   0               0        0   
                           R4                   0               1        1   
                           R5                   0               0        1   

                                  goal_assists  %_played  
team     year player       round                          
adelaide 2019 Atkins, Rory R1                0        88  
                           R2                0        87  
                           R3                0        77  
                           R4                0        87  
                           R5                1        85

In [8]:
df

disposals  kicks  marks  handballs  \
team      year player            round                                       
adelaide  2019 Atkins, Rory      R1            31     22      7          9   
                                 R2            21     14      3          7   
                                 R3            12      6      3          6   
                                 R4            21     17      6          4   
                                 R5            26     17      5          9   
                                 R6            27     17      6         10   
                                 R7            18      9      2          9   
                                 R8            21     17      7          4   
               Betts, Eddie      R1            10      6      3          4   
                                 R2             9      7      1          2   
                                 R3            15      8      2          7   
                                 R4            11     10      3          1   
                                 R5            14     11      1          3   
                                 R6            10      7      3          3   
                                 R7            12      7      2          5   
                                 R8            10      7      1          3   
               Brown, Luke       R8            18     12      4          6   
               Crouch, Brad      R1            28     13      6         15   
                                 R2            26     15      5         11   
                                 R3            35     18      4         17   
                                 R4            24      7      2         17   
                                 R5            36     13      1         23   
                                 R6            30     17      5         13   
                                 R7            31     11      2         20   
                                 R8            30     10      1         20   
               Crouch, Matt      R1            40     16      5         24   
                                 R2            24     12      5         12   
                                 R3            27     12      2         15   
                                 R4            29     15      6         14   
                                 R5            36     13      5         23   
...                                           ...    ...    ...        ...   
bullldogs 2019 Suckling, Matt    R3            19     14      6          5   
                                 R4            27     21      8          6   
                                 R8            21     14      4          7   
               Trengove, Jackson R6            12      7      4          5   
                                 R7            14      9      5          5   
                                 R8            21     11      2         10   
               Wallis, Mitch     R1            26     10      4         16   
                                 R2            29     13      5         16   
                                 R3            23     12      2         11   
                                 R4            19      8      4         11   
                                 R5            16      7      1          9   
                                 R6            21      9      3         12   
                                 R7            23     10      6         13   
                                 R8            17     10      3          7   
               Williams, Bailey  R1            12     11      4          1   
                                 R5             9      5      3          4   
               Wood, Easton      R1             5      5      3          0   
                                 R2             9      7      4          2   
                                 R3            12      9      